IMPORTANT NOTE
- if mo ask si maam why this is similar to mine and mag suspect siya na i made it, kay just tell her na youre using mediapipe instead of tensorflow
- 

flaw/weaknesses observed:

evident luminance affecting performance, resulting to erratic tracking which causing it to abruptly stop collecting and considering it as one execution
- should always perform/collect data at a well lit environment
- should apply frame patience mechanism wherein it ignores certain amount of frames before it consider as an execution [APPLIED!]

overload of data
- apply frame buffering(less frame already is sufficient enough to resemble the movement)
- eliminate z coordinate(pose estimation is positional dependant)
- experiment on lessening decimal places (this would sacrifice a certain amount of accuracy, should heavily consider how much of it)


future planned features
 - real-time frame by frame visualization every after execution to check
 - easy-delete button wherein it deletes the current execution collected in case its not good or intended 


How data collection works?
1) show your hands on the camera----------------> data collection starts
2) perform the action you want -----------------> coordinates are collected(coordinates_frame are generated and are compiled in coordinates_movemenet )
3) remove your hand from the camera ------------> data collection stops(coordinates_movemenet is compiled in execution)
4) repeat...

coordinates
- individual X and Y coordinates tracked and collected 
- how it looks like: 0.435. 0.454....etc


coordinates_frame
- compilation of coordinates
- how it looks like: [coordinates, coordinates, coordinates, coordinates..., coordinates,]

execution
- compilation of coordinates_frame, which resembles a movement 
- how it looks like:
[
[coordinates, coordinates, coordinates, coordinates..., coordinates,],
[coordinates, coordinates, coordinates, coordinates..., coordinates,],
...
[coordinates, coordinates, coordinates, coordinates..., coordinates,],
]

datasets
- compilation of executions
- how it looks like:
[
execution,
execution,
execution,
...
execution,
]


In [98]:
import cv2
import mediapipe as mp


import os
import numpy as np
import tkinter as tk
from tkinter import simpledialog
from threading import Thread
import os

import random
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical




In [99]:

# Globals
detection_mode = "Multiple"
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
label_map = {}
trained = False

# MediaPipe for hand tracking
mp_hands = mp.solutions.hands
hands_detector = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

known_folder = os.path.join(os.getcwd(), "datasets/")
parent_folder = os.path.dirname(known_folder)
file_path = os.path.join(parent_folder, "output.txt")

cap = cv2.VideoCapture(1)

data_pad = []



In [ ]:

def pose_estimation():
    frame_buffer= 8
    frame_buffer_ctr = 0

    frame_patience = 8
    frame_patience_ctr = 0
    
    coordinates_frame = []
    coordinates_movemenet = []
    execution = []
    initialize_padding()

    while True:        
        frame_buffer_ctr+=1
        if frame_buffer_ctr>=frame_buffer:   
            frame_buffer_ctr = 0         
            ret, frame = cap.read()
            if not ret:
                break

            frame = cv2.flip(frame, 1)
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands_detector.process(rgb)

            if results.multi_hand_landmarks:    
                frame_patience_ctr=0

            if results.multi_hand_landmarks and results.multi_handedness:

                detected_hands = [hand.classification[0].label for hand in results.multi_handedness]

                for hand_landmarks, hand_handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
                    hand_label = hand_handedness.classification[0].label 
                    print(f"Detected {hand_label} hand -> {len(coordinates_frame)}")
                    mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                    for idx, landmark in enumerate(hand_landmarks.landmark):
                        coordinates_frame.append(f"{landmark.x:.3f}")
                        coordinates_frame.append(f"{landmark.y:.3f}")

                if 'Left' in detected_hands and 'Right' not in detected_hands and len(coordinates_frame) != 84:
                    coordinates_frame.extend(data_pad)

                elif 'Right' in detected_hands and 'Left'not in detected_hands  and len(coordinates_frame) != 84:
                    temp = coordinates_frame
                    coordinates_frame = []                    
                    coordinates_frame.extend(data_pad)
                    coordinates_frame.extend(temp)
                     
                coordinates_movemenet.append(coordinates_frame)
                coordinates_frame=[]
            else:
                if frame_patience_ctr<=frame_patience:
                    frame_patience_ctr += 1

                if frame_patience_ctr >= frame_patience:
                    if len(coordinates_movemenet) != 0:    
                        execution.append(coordinates_movemenet)
                        print("execution collected -> ", len(execution),"[",len(coordinates_movemenet) ,"]")

                        coordinates_movemenet = []

            cv2.imshow("Recording Face and Hand", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()  
    return execution


def inferencing():
    frame_buffer= 8
    frame_buffer_ctr = 0

    frame_patience = 8
    frame_patience_ctr = 0
    
    coordinates_frame = []
    coordinates_movemenet = []
    execution = []
    initialize_padding()

    while True:        
        frame_buffer_ctr+=1
        if frame_buffer_ctr>=frame_buffer:   
            frame_buffer_ctr = 0         
            ret, frame = cap.read()
            if not ret:
                break

            frame = cv2.flip(frame, 1)
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands_detector.process(rgb)

            if results.multi_hand_landmarks:    
                frame_patience_ctr=0

            if results.multi_hand_landmarks and results.multi_handedness:

                detected_hands = [hand.classification[0].label for hand in results.multi_handedness]

                for hand_landmarks, hand_handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
                    hand_label = hand_handedness.classification[0].label 
                    print(f"Detected {hand_label} hand -> {len(coordinates_frame)}")
                    mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                    for idx, landmark in enumerate(hand_landmarks.landmark):
                        coordinates_frame.append(f"{landmark.x:.3f}")
                        coordinates_frame.append(f"{landmark.y:.3f}")

                if 'Left' in detected_hands and 'Right' not in detected_hands and len(coordinates_frame) != 84:
                    coordinates_frame.extend(data_pad)

                elif 'Right' in detected_hands and 'Left'not in detected_hands  and len(coordinates_frame) != 84:
                    temp = coordinates_frame
                    coordinates_frame = []                    
                    coordinates_frame.extend(data_pad)
                    coordinates_frame.extend(temp)
                     
                coordinates_movemenet.append(coordinates_frame)
                coordinates_frame=[]
            else:
                if frame_patience_ctr<=frame_patience:
                    frame_patience_ctr += 1

                if frame_patience_ctr >= frame_patience:
                    if len(coordinates_movemenet) != 0:    
                        execution.append(coordinates_movemenet)
                        print("execution collected -> ", len(execution),"[",len(coordinates_movemenet) ,"]")

                        coordinates_movemenet = []

            cv2.imshow("Recording Face and Hand", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()  
    return execution




def text_encode(coordinates_data):
    with open(file_path, "w") as file:
        for execution in coordinates_data:
            file.write("START\n")
            for movement_coordinates in execution:
                for coordinate in movement_coordinates:
                    file.write(str(coordinate))
                    file.write("|")
                file.write("\n")
            file.write("END\n")


def text_decode(file_path):
    coordinates_data = []
    with open(file_path, "r") as file:
        lines = file.readlines()

    execution = []
    for line in lines:
        line = line.strip()
        if line == "START":
            execution = []
        elif line == "END":
            coordinates_data.append(execution)
        else:
            str_coords = line.split("|")[:-1]
            float_coords = [float(coord) for coord in str_coords]
            execution.append(float_coords)

    return coordinates_data



def initialize_dummy_dataset():
    frame_coordinate = []
    movement_coordinate = []
    executions = []

    for x in range(60):
        frame_coordinate.append(0.69)

    for i in range(random.randint(8, 15)):    
        for x in range(random.randint(8, 15)):
            movement_coordinate.append(frame_coordinate)
        executions.append(movement_coordinate)
        movement_coordinate = []

    return executions

def initialize_padding():
    if len(data_pad) != 42:
        for ctr in range(42):
            data_pad.append(0.0);
        print(f"len of data pad {len(data_pad)}")



def decode_all_txt_files(folder_path):
    all_data = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            data = text_decode(file_path)
            all_data.append(data) 

    return all_data

def data_padding(input_datasets):
    initialize_padding()
    datasets = input_datasets
    max_len = 0
    padding = []
    padding.extend(data_pad)
    padding.extend(data_pad)
    print(len(data_pad))


    for classifications in datasets:
        for executions in classifications:
            if max_len <= len(executions):
                max_len = len(executions)

    for classification in datasets:
        for executions in classification:
            while(len(executions) < max_len):
                executions.append(padding)

    return datasets

def data_labelling(input_dataset):
    temp_input_dataset = input_dataset
    label = []
    data = []
    for ctr in range(len(temp_input_dataset)):
        for ctr2 in range(len(temp_input_dataset[ctr])):
            label.append(ctr)
            data.append(temp_input_dataset[ctr][ctr2])
    return [data,label]



# def coordinates_visualization(coordintes_dataset):
#     canvas_size = 500
#     canvas = np.zeros((canvas_size, canvas_size * 2, 3), dtype=np.uint8)

#     # for execution in coordintes_dataset:
#         # for frame_coordinates in execution[0]:
#         #     # for individual_coordinates in frame_coordinates:
#         #     for individual_coordinates in range(len(frame_coordinates)):
#         #         if individual_coordinates + 1 < len(frame_coordinates):
#         #             if individual_coordinates % 2 != 0:
#         #                 cv2.circle(canvas, (int(frame_coordinates[individual_coordinates]*1000),int(frame_coordinates[individual_coordinates+1]*1000)), radius=4, color=(0, 255, 0), thickness=-1)
#     print(len(coordintes_dataset[0]))
#     for frame_coordinates in coordintes_dataset[0]:
#         # for individual_coordinates in frame_coordinates:
#         for individual_coordinates in range(len(frame_coordinates)):
#             if individual_coordinates + 1 < len(frame_coordinates):
#                 if individual_coordinates % 2 != 0:
#                     cv2.circle(canvas, (int(frame_coordinates[individual_coordinates]*1000),int(frame_coordinates[individual_coordinates+1]*1000)), radius=4, color=(0, 255, 0), thickness=-1)

#     cv2.imshow("Painted Coordinates", canvas)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
#     return 0











In [ ]:
# collecting dataset
text_encode(pose_estimation())

In [103]:
temp = decode_all_txt_files(parent_folder)
temp_padded = data_padding(temp)
label = data_labelling(temp_padded)
data_shuffled, label_shuffled = shuffle(label[0], label[1], random_state=42)
data_shuffled = np.array(data_shuffled)
label_shuffled = np.array(label_shuffled)



42


In [ ]:
num_classification = 2

X_train, X_test, y_train, y_test = train_test_split(
    data_shuffled, label_shuffled, test_size=0.2, random_state=42
)

model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(data_shuffled.shape[1], data_shuffled.shape[2])))
model.add(Dropout(0.3))
model.add(LSTM(64))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classification, activation='softmax'))  # num_classes

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=30, batch_size=8, validation_data=(X_test, y_test))
model.save("my_model.h5")  # or use .keras for newer format


Epoch 1/30
13/13 [==============================] - 7s 155ms/step - loss: 0.6893 - accuracy: 0.5773 - val_loss: 0.6631 - val_accuracy: 0.5600
Epoch 2/30
13/13 [==============================] - 0s 24ms/step - loss: 0.6442 - accuracy: 0.6289 - val_loss: 0.9001 - val_accuracy: 0.6400
Epoch 3/30
13/13 [==============================] - 0s 21ms/step - loss: 0.6400 - accuracy: 0.6598 - val_loss: 0.6371 - val_accuracy: 0.6400
Epoch 4/30
13/13 [==============================] - 0s 20ms/step - loss: 0.5510 - accuracy: 0.7835 - val_loss: 0.4984 - val_accuracy: 0.8400
Epoch 5/30
13/13 [==============================] - 1s 42ms/step - loss: 0.3988 - accuracy: 0.7938 - val_loss: 0.3800 - val_accuracy: 0.8800
Epoch 6/30
13/13 [==============================] - 0s 29ms/step - loss: 0.3224 - accuracy: 0.9072 - val_loss: 0.2659 - val_accuracy: 0.9200
Epoch 7/30
13/13 [==============================] - 0s 21ms/step - loss: 0.3833 - accuracy: 0.8660 - val_loss: 0.3597 - val_accuracy: 0.8800
Epoch 8/30
1